# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

In [1]:
# !pip install ipywidgets

In [2]:
# !pip install textdistance

In [3]:
# !pip install razdel tqdm

In [4]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
import textdistance
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [5]:
from tqdm.notebook import tqdm

In [6]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

In [7]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [8]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [9]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [10]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)
    vars2sort = similarities.most_common(topn)
    
    words, sims = zip(*vars2sort)
    visited = set()
    dupl = set(x for x in sims if x in visited or (visited.add(x) or False))
    # список повторяющихся расстояний редактирования

    similarities_freqcorrected = Counter()  
    got_similarity = None
    index = 1
    word_freq = Counter() # для слов с повторяющимся расстоянием редактирования: слово-частотность
    for word, similarity in vars2sort:
        if similarity in dupl:
            word_freq[word] = P(word)
            # логарифм вероятности не использую, потому что и так занулиться ничего не должно
        else:
            for word_to_sort in word_freq.most_common():
                similarities_freqcorrected[word_to_sort[0]] = index
                index+=1
            word_freq = Counter()
            similarities_freqcorrected[word] = index
            index+=1
    if word_freq != Counter():
        for word_to_sort in word_freq.most_common():
                similarities_freqcorrected[word_to_sort[0]] = index
                index+=1

    return similarities_freqcorrected.most_common()[:-topn-1:-1]

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    return closest

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [11]:
get_closest_hybrid_match('сонце', X, vec)
# Мы тут не сохранили вывод расстояния редактирования, но нас и не просили

[('солнце', 1), ('соне', 2), ('сосновец', 3)]

Оценим качество

In [12]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [13]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)
# Без добавления ранжирования по вероятности качество такое
# 0.847023511755878
# 0.421583850931677
# 0.09004249454461927

0.8477238619309655
0.42701863354037267
0.09004249454461927


Результат улучшился, но незначительно

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [14]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))  # Составляется словарь правильных слов

Удалять можно ровно 1 символ.

In [15]:
corrections = {}
for word in vocab:
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    for delete in deletes:
        if delete in corrections:
            corrections[delete].append(word)
        else:
            corrections[delete] = [word]
# Составляется словарь удалений
# по условию может быть несколько вариантов правильных слов для слова с опечаткой,
# поэтому  у нас не "ключ - слово с удалением, а значение - правильное слово",
# а "ключ - слово с удалением, а значение - список правильных слов"

In [16]:
def get_corrected(word):
    if word in corrections:
        if len(corrections[word]) > 1:
            word_freq = Counter()
            for var in corrections[word]:
                word_freq[var] = P(var)
            corrected = word_freq.most_common(1)[0][0]
        else:
            corrected = corrections[word][0]
    else:
        corrected = word
    return corrected

In [17]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_corrected(pair[1]))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                print(pair[0], pair[1], pred)
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

получатся полчатся получатся
отсутствие отсуствие отсутствие
основная основая основная
напрасно нарасно напрасно
предлагаю предлагю предлагаю
сегодняшнее сегодяшнее сегодняшнее
лучше лчше лучше
компьютерная компютерная компьютерная
ответственность отвественность ответственность
комментариев коментариев комментариев
ответственный отвественный ответственный
рассчитаны расчитаны рассчитаны
расспрашивая распрашивая расспрашивая
странно сранно странно
вообще вобще вообще
зубные зуные зубные
сочувствующие сочуствующие сочувствующие
рассказчик расказчик рассказчик
приехали прихали приехали
немного нмного немного
пьяный пяный пьяный
священнослужители священослужители священнослужители
первый первй первый
вообще вобще вообще
гулливера гуливера гулливера
одноклассниках однокласниках одноклассниках
температура тепература температура
симптомы симтомы симптомы
потому потму потому
одноклассников однокласников одноклассников
профессор професор профессор
сегодняшние сегодяшние сегодняшние
рассмотрим р

In [18]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8818409204602301
0.15993788819875776
0.011370161938670036


Нужно словарь больше брать. И токенизировать лучше. И align_words поправить. У нас вот такие куски в mistakes встречаются:

    ('насчет', 'нащщот', 'нащщот'),
    ('в', 'вобщем', 'вобщем'),
    ('общем', 'как', 'как'),
    ('как', 'вы', 'вы'),
    ('вы', 'знаете', 'знаете'),
    ('знаете', 'из', 'из'),
    ('из', 'моего', 'моего'),
    ('моего', 'не', 'не'),
    ('недавнего', 'давнего', 'давнего'),

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

BPE - алгоритм составления словаря символов для дальнейшей кодировки токенов.

Можно назначать кодировку отдельным словам. Но что делать, если встретилось новое слово, которого не было, например, в тренировочной выборке? Можно попробовать всем-всем словам в мире назначить кодировку. Но тогда у нас будет бесконечный словарь. Держать кодировку для неизвестных слов? Можно назначить кодировку каждой букве. Тогда мы сможем кодировать любые слова, но получится, что буквы независимы друг от друга, а мы знаем, что какие-то сочетания встречаются чаще, у некоторых сочетаний есть понятные значения (у корней слов или других морфем, например).

Поэтому можно использовать BPE. Сначала у нас в словаре буквы, которые встретились в тексте (+ символ конца слова, чтобы у нас не получились символы длиннее слов). Два символа, которые чаще всего встречаются вместе, объединим в сочетание - единый символ. И дальше будем объединять самые часто встречающиеся пары в отдельные символы. Благодаря этому, остановившись в любой момент, мы получим словарь символов, которыми можно закодировать любой текст (на знакомом нам языке, со знакомым нам алфавитом). Частотные слова будут для нас едиными символами целиком. Неизвестные слова разобьются на известные части, морфемы и в крайнем случае буквы. Для кодировки самых частотных сочетаний мы будем использовать меньше информации, чем нужно для побуквенной кодировки этих сочетаний. Наша кодировка будет осмысленнее, потому что морфемы и целые слова содержат больше значения, чем буквы.

P.S. Вы разрешили позже сдать. Я почти весь декабрь и новогодние каникулы болела (справка есть). Простите...